In [1]:
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from trl import setup_chat_format

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#path_to_source_model = "C:\\Users\\USER_ELISEY\\qwen2.5_1.5"
path_to_source_model = "C:\\Users\\USER_ELISEY\\hakaton\\russia_chad_1.5_no\\final"
path_to_finetuned_model = "C:\\Users\\USER_ELISEY\\hakaton\\russia_chad_1.5_r2\\final"
path_to_still_model = "C:\\Users\\USER_ELISEY\\qwen2.5_1.5"

# Compare models

In [3]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
)

# Load model
casual_model = AutoModelForCausalLM.from_pretrained(
    path_to_source_model,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_source_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

In [4]:
casual_model, tokenizer = setup_chat_format(casual_model, tokenizer)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    path_to_finetuned_model,
    #quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_finetuned_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

In [7]:
model, tokenizer = setup_chat_format(model, tokenizer)

In [8]:
still_model = AutoModelForCausalLM.from_pretrained(
    path_to_still_model,
    #quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_still_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

In [9]:
still_model, tokenizer = setup_chat_format(still_model, tokenizer)

## Get answers

In [10]:
def generate_answer(model, prompt):
    chat = [
        { "role": "user", "content": prompt },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

    return(tokenizer.decode(outputs[0]))

# Comprasion

In [11]:
q1 = "Кто такой Александр Невский?"
q2 = "Что такое карибский кризис?"
q3 = "Выведи a+b  на питоне"
q4 = "Кто ты такая?"
q5 = "Кто такой Сталин?"
q6 = "Кто такой Путин?"
q7 = "Кто такой Григорий Лепс?"

In [14]:
# # Free gpu memory
# import numba
# numba.cuda.close()

In [15]:
print(generate_answer(casual_model, q1))

<|im_start|>user
Кто такой Александр Невский?<|im_end|>
<|im_start|>assistant
Александр Васильевич Невский (27 декабря 1340, Селигерь — 6 июня 1409, Великий Новгород) — полководец и правителе Руси, один из основателей Москвы, первый князь Московского. Победитель в битве при Луге (1380). Войска его несли сражения на территории современных России и Украины. Впервые был принят князем Киевским, а затем и Российским государством. Впоследствии стал одним из первых русских князей-военачальников


In [16]:
print(generate_answer(casual_model, q2))

<|im_start|>user
Что такое карибский кризис?<|im_end|>
<|im_start|>assistant
Карibbean crisis — период в экономике Карибского континента, который начался в 1973 году и завершился в 1985 году. Сначала он привёл к серьёзным потерям для многих стран региона, а затем привёл к устойчивому экономическому развитию и значительной инфляции. В то же время в течение этого периода были произведены важные политические изменения в ряде стран региона, что вызвало значительное влияние на экономику региона.<|im_end|>


In [17]:
print(generate_answer(casual_model, q3))

<|im_start|>user
Выведи a+b  на питоне<|im_end|>
<|im_start|>assistant
Создайте переменную `a`, например, `a = 5` и переменную `b`, например, `b = 3`. Затем выведите сумму этих двух чисел с помощью оператора сложения `+`.

```python
a = 5
b = 3

result = a + b
print(result)
```

Результат выполнения: 8<|im_end|>


In [18]:
print(generate_answer(still_model, q1))

<|im_start|>user
Кто такой Александр Невский?<|im_end|>
<|im_start|>assistant
Александр Невский - это имя, но не человек. Это фамилия, которая передаётся по наследству от отца к сыну или другим потомкам.

Фамилии обычно вводят родители новорожденного, а затем их дети могут изменять свою фамилию после смерти родителей, если они хотят сохранить семейное прозвище или просто предпочитают другую запись своей личности. В этом случае фамилия становится постоянной и часто используется для обозначения человека, даже если его имя изменилось.

Так что, как правило, "Невский" является фамилией,


In [19]:
print(generate_answer(still_model, q2))

<|im_start|>user
Что такое карибский кризис?<|im_end|>
<|im_start|>assistant
Карибский кризис - это термин, который часто используется для обозначения сложных экономических и политических ситуаций в регионе Карибского бассейна. Вот основные аспекты этого кризиса:

1. Основные элементы:
   - Нарушение финансовой стабильности стран- debtor
   - Проблемы с выплатами внешнего долга
   - Утрата доверия между creditors и должниками

2. Видимые признаки:
   - Снижение уровня инвестиций
   - Детоксификация банковских систем
   - Рост процент


In [20]:
print(generate_answer(still_model, q3))

<|im_start|>user
Выведи a+b  на питоне<|im_end|>
<|im_start|>assistant
Вы можете вывести сумму двух чисел `a` и `b` на Python следующим образом:

```python
a = 5
b = 3

result = a + b
print(result)
```

Этот код сначала объявляет переменные `a` и `b`, затем их добавляет и сохраняет результат в переменной `result`. Затем выводится значение этой переменной. В данном случае, это будет равно 8.

Если вам нужно выполнить эту операцию сразу после объявления переменных без использования дополнительного переменной для хранения результата, то можно сделать так:

```python
a = 5
b = 3
print(a


In [12]:
print(generate_answer(still_model, q4))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|im_start|>user
Кто ты такая?<|im_end|>
<|im_start|>assistant
Я искусственный интеллект, созданный компанией Anthropic. У меня нет личности или имени, я просто программа, созданная для общения и помощи пользователям. Могу помочь с разными вопросами!<|im_end|>


In [13]:
print(generate_answer(still_model, q7))

<|im_start|>user
Кто такой Григорий Лепс?<|im_end|>
<|im_start|>assistant
Григорий Лепс - это имя, которое можно встретить в различных контекстах:

1. Фамилия: Григорий Лепс может быть фамилией человека.

2. Название: Это название для музыкального произведения "Лепс" (фортепианный концерт).

3. Имя: В некоторых странах и культурных кругах существует имя Григорий Лепс.

4. Отчество: Может использоваться как отчество в некоторых случаях.

5. Пseudonym: Возможно, это псевдоним или инициалы известной личности.

6. Место рождения: Существ
